## imports and files

In [1]:
import pandas as pd
import os
import sys
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import PIPELINE_PROFILING_RESULTS_PATH
from experiments.utils.parser import Parser

In [2]:
series = 4
config_key_mapper = "key_config_mapper.csv"

series_path = os.path.join(PIPELINE_PROFILING_RESULTS_PATH, "series", str(series))

loader = Parser(
    series_path=series_path,
    config_key_mapper=config_key_mapper,
    model_name="video",
    type_of="router_pipeline",
)

In [3]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 0.yaml
{   'benchmark_duration': 1,
    'central_queue': False,
    'distrpution_time': 15,
    'metadata': 'pipeline benchmarking',
    'mode': 'exponential',
    'nodes': [   {   'cpu_request': ['1'],
                     'data_type': 'image',
                     'max_batch_size': ['1'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['yolov5n'],
                     'node_name': 'yolo',
                     'num_interop_threads': ['1'],
                     'num_threads': ['1'],
                     'replicas': [1],
                     'use_threading': 'True'},
                 {   'cpu_request': ['1'],
                     'data_type': 'text',
                     'max_batch_size': ['4'],
                     'max_batch_time': ['1'],
                     'memory_request': ['10Gi'],
                     'model_variants': ['resnet18'],
                     'node_name': 'resnet-human',


## config map reader

In [5]:
key_config_df = loader.key_config_mapper()
# display(key_config_df)
key_config_df.columns
results = loader.result_processing()

In [15]:
results

""


In [16]:
for item in results.columns:
    print(item)

In [17]:
experiment_id = 1
experiment_results = loader.get_experiment_detail(experiment_id=experiment_id)
# display(experiment_results)
# -------------------------
experiment_results.keys()
filenames = loader.get_result_file_names()
# display(filenames)
# -------------------------
results = loader._read_results()
# display(results.keys())
# -------------------------
flattened_results = loader.flatten_results(results["1"]["responses"])
# display(flattened_results)
# -------------------------
experiment_id = 1
_, flattened_results = loader.flatten_results(results[str(experiment_id)]["responses"])
latencies, timeout_count = loader.latency_calculator(flattened_results)
# display(pd.DataFrame(latencies))
# -------------------------
results = loader.result_processing()
# print(results)
# print(list(results.columns))
for item in list(results.columns):
    if "task_0" in item:
        print(item)
# -------------------------
for item in results.columns:
    print(item)
# -------------------------
# experiment_id = 1
# _, flattened_results = loader.flatten_results(results[str(experiment_id)]["responses"])
# latencies, timeout_count = loader.latency_calculator(flattened_results)
# display(pd.DataFrame(latencies))
# -------------------------
# summary = loader.latency_summary(latencies=latencies)
# pd.DataFrame(list(summary.items()))

In [18]:
key_config_df = loader.key_config_mapper()
print(key_config_df.columns)
# selected_key_config_df =  key_config_df[key_config_df['model_variant_1'] == 'yolov5n']
# selected_key_config_df
key_config_df

Index(['experiment_id', 'pipeline_name', 'load', 'load_duration', 'series',
       'metadata', 'mode', 'data_type', 'benchmark_duration',
       'task_0_node_name', 'task_0_model_variant', 'task_0_cpu_request',
       'task_0_memory_request', 'task_0_max_batch_size',
       'task_0_max_batch_time', 'task_0_replica', 'task_1_node_name',
       'task_1_model_variant', 'task_1_cpu_request', 'task_1_memory_request',
       'task_1_max_batch_size', 'task_1_max_batch_time', 'task_1_replica'],
      dtype='object')


,experiment_id,pipeline_name,load,load_duration,series,metadata,mode,data_type,benchmark_duration,task_0_node_name,...,task_0_max_batch_size,task_0_max_batch_time,task_0_replica,task_1_node_name,task_1_model_variant,task_1_cpu_request,task_1_memory_request,task_1_max_batch_size,task_1_max_batch_time,task_1_replica
0,1,video,10,60,3,pipeline benchmarking,exponential,image,1,yolo,...,1,1,1,resnet-human,resnet18,1,10Gi,4,1,1


In [19]:
experiment_ids = key_config_df[key_config_df["load"] == 10]["experiment_id"].tolist()

metadata_columns = [
    "experiment_id",
    "task_0_cpu_request",
    "task_0_model_variant",
    "task_0_replica",
    "task_0_max_batch_size",
    "task_1_cpu_request",
    "task_1_model_variant",
    "task_1_replica",
    "task_1_max_batch_size",
]

selected_results_columns = [
    "client_to_pipeline_avg",
    "task_0_model_avg",
    "task_0_to_task_1_latencies_avg",
    "task_1_model_avg",
]


output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=selected_results_columns,
)
pd.set_option("display.max_rows", None)
output["model_latencies_sum"] = output["task_0_model_avg"] + output["task_1_model_avg"]
output["e2e_latencies"] = (
    output["task_0_model_avg"]
    + output["task_1_model_avg"]
    + output["task_0_to_task_1_latencies_avg"]
)

KeyError: 'experiment_id'

In [ ]:
display(
    output[output["task_0_to_task_1_latencies_avg"] > 0]
    .sort_values(by=["model_latencies_sum"], ascending=False)
    .head(10)
)

,experiment_id,task_0_cpu_request,task_0_model_variant,task_0_replica,task_0_max_batch_size,task_1_cpu_request,task_1_model_variant,task_1_replica,task_1_max_batch_size,client_to_pipeline_avg,task_0_model_avg,task_0_to_task_1_latencies_avg,task_1_model_avg,model_latencies_sum,e2e_latencies
0,1,1,yolov5n,1,1,1,resnet18,1,4,0.51474,0.082847,0.043527,0.196241,0.279088,0.322615


In [ ]:
a = (
    output[output["task_0_to_task_1_latencies_avg"] > 0]
    .sort_values(by=["model_latencies_sum"], ascending=False)
    .experiment_id.to_numpy()
)
a

array([1])

In [ ]:
b = (
    output[output["task_0_to_task_1_latencies_avg"] > 0]
    .sort_values(by=["e2e_latencies"], ascending=False)
    .experiment_id.to_numpy()
)
b

array([1])

In [ ]:
a

array([1])

In [ ]:
b

array([1])

In [ ]:
sum(a == b)

1